In [489]:
import numpy as np
from math import sqrt, ceil, pi, sin, cos, tan
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, interp2d
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
np.set_printoptions(precision=2, suppress=True)
%precision 2


P = 7 # kW
n = 65 # rpm
L = 8 # years
Kng = 260 # days
Ca = 1 # shifts
t1 = 15 # s
t2 = 11 # s
T1 = 1 # N.mm
T2 = .7 # N.mm

def odd(num):
    if num%2==0: return num+1
    else: return num
def disp(*args):
    a = []
    for arg in args:
        a.append(arg)
    return np.array(a,dtype=float)

fkses1 = interp1d([400,500,600,700,800,900,1000,1200],
            [2.25,2.5,2.75,3,3.25,3.5,3.75,4.25])
fkses2 = interp1d([400,500,600,700,800,900,1000,1200],
             [2.75,3.05,3.36,3.66,3.96,4.28,4.6,5.2])
fkses1 = interp1d([400,500,600,700,800,900,1000,1200],
            [2.25,2.5,2.75,3,3.25,3.5,3.75,4.25])
fkses2 = interp1d([400,500,600,700,800,900,1000,1200],
             [2.75,3.05,3.36,3.66,3.96,4.28,4.6,5.2])
fes = interp1d([15,20,30,40,50,70,80,100],
              [0.87,0.83,0.77,0.73,0.7,0.66,0.64,0.62])
fet = interp1d([15,20,30,40,50,70,80,100],
               [0.92,0.89,0.81,0.78,0.76,0.73,0.71,0.7])

# fes(850),fes(850)

## Motor Design $(power, rpm)$

In [490]:
# calculate efficiency, e = eta
e_c = 1
e_b = 0.99
e_hg = 0.97 # helical gear
e_ch = 0.96
e_sys = e_c * e_b**4 * e_hg**2 * e_ch
e_sys

0.87

In [491]:
P_max = P # calculate max power of the belt conveyor, kW
P_w = P_max*sqrt((T1**2*t1 + T2**2*t2)/(t1+t2)) # calculate power given the work load, kW
P_mo = P_w/e_sys # lower limit of the motor's power, kW
P_max, P_w, P_mo # compare P_max & P_mo, choose larger value to set lower limit and look up the catalogue

(7, 6.20, 7.14)

In [492]:
u_ch = 4
u_h = 10
u_sys = u_ch * u_h # calculate transmission ratio of the system
n_mo = u_sys * n # calculate theoretical rotational speed of the motor, rpm
n, u_sys, n_mo

(65, 40, 2600)

In [493]:
P_mo = 7.5 # power based on choice of motor, kW
n_mo = 2922 # rotational speed based on choice of motor, rpm
u_sys = n_mo / n # new transmission ratio
u_ch = u_sys / u_h # new transmission ratio of ch drive, given u_hg=const
u_sys, u_ch

(44.95, 4.50)

In [494]:
P_ch  = P_w / e_b
P_sh3 = P_ch / e_ch # power of shaft 3, kW
P_sh2 = P_sh3 / (e_b*e_hg)
P_sh1 = P_sh2 / (e_b*e_hg)
P_mo, P_sh1, P_sh2, P_sh3

(7.50, 7.07, 6.79, 6.52)

In [495]:
u_1 = .7332*u_h**.6438
u_2 = u_h/u_1
n_sh1 = n_mo # rotational speed of shaft 1, rpm
n_sh2 = n_sh1 / u_1
n_sh3 = n_sh2 / u_2
u_1, u_2, n_sh1, n_sh2, n_sh3

(3.23, 3.10, 2922, 905.02, 292.20)

In [496]:
T_mo  = 9.55e6 * P_mo  / n_mo # torque of motor, N.mm
T_sh1 = 9.55e6 * P_sh1 / n_sh1 # torque of shaft 1, N.mm
T_sh2 = 9.55e6 * P_sh2 / n_sh2
T_sh3 = 9.55e6 * P_sh3 / n_sh3
T_mo, T_sh1, T_sh2, T_sh3

(24512.32, 23116.54, 71672.45, 213175.22)

## Chain Drive Design

In [497]:
# find z1, z2
n1 = n_sh3
z1 = ceil(29 - 2*u_ch)
z2 = odd(ceil(u_ch*z1)) # eq (5.1), p.80, obtain odd number of teeth
z1, z2

(21, 95)

In [498]:
# find P_t
n01 = 200 # ~ n_sh2
k_z, k_n = 25/z1, n01/n1
k_0, k_a, k_dc, k_bt, k_d, k_c = 1, 1, 1, 1, 1.25, 1
k = k_0 * k_a * k_dc * k_bt * k_d * k_c # table 5.6, p.82
P_t = P_ch * k * k_z * k_n
k_z, k_n, P_t

(1.19, 0.68, 6.38)

In [499]:
# find P_allowance and [a_min, a_max]
P_allowance, p, dc, B = 11, 25.4, 7.95, 22.61 # table 5.5, p.81
a_min, a_max = 30*p, 50*p
a_min, a_max

(762.00, 1270.00)

In [500]:
a = 1000 # mm
x = 2*a/p + (z1+z2)/2 + (z2-z1)**2*p/(4*pi**2*a) # eq (5.12), p.85
xc = odd(ceil(x)) + 1
a = p/4*(xc - 0.5*(z2+z1) + sqrt((xc-0.5*(z2+z1))**2-2*((z2-z1)/pi)**2)) * (1-.003)
i = z1*n1/(15*x)
x, xc, a, i

(140.26, 142, 1019.99, 2.92)

In [501]:
k_d, dl, Q, q, k_f = 1.2, 15.88, 56.7, 2.6, 6 # table 5.2; eq 5.15
fs = interp1d([50,200,400,600,800,1e3],[7,8.2,9.3,10.3,11.7,12.9]) # table 5.10
v1 = n1*p*z1/6e4
Ft, Fv, F0 = 1e3*P_ch/v1, q*v1**2, 9.81*k_f*q*a/1e3
s = Q*1e3/(k_d*Ft + F0 + Fv)
s_allowable = float(fs(n1))
v1, Ft, Fv, F0, s, s_allowable

(2.60, 2410.48, 17.54, 156.10, 18.49, 8.71)

In [502]:
sigHm, E, A, k_d = 600, 2.1e5, 180, 1.2
Fvd = 13e-7*n1*p**3
fkr = interp1d([15,20,30,40,50,60],[.59,.48,.36,.29,.24,.22])
k_r = float(fkr(z1))
sigH=.47*sqrt(k_r*(Ft*k_d+Fvd)*E/A)
Fvd, k_r, sigH, sigHm

(6.22, 0.47, 591.29, 600)

In [503]:
k_x = 1.15
F2 = F0+Fv
F1 = Ft+F2
Fr = k_x*Ft
F2, F1, Fr

(173.64, 2584.12, 2772.05)

In [504]:
d1 = p/sin(pi/z1)
d2 = p/sin(pi/z2)
da1 = p*(.5 + 1/tan(pi/z1))
da2 = p*(.5 + 1/tan(pi/z2))
df1 = d1 - 2*(.5025*dl+.05)
df2 = d2 - 2*(.5025*dl+.05)
d1, d2, da1, da2, df1, df2

(170.42, 768.22, 181.22, 780.50, 154.36, 752.16)

## Gear design

In [505]:
HB, sigb, sigch = 250, 850, 550 # MPa
SH, SF = 1.1, 1.75
H1, H2 = HB, HB-10
sigHlim1, sigFlim1 = 2*H1+70, 1.8*H1
sigHlim2, sigFlim2 = 2*H2+70, 1.8*H2
H1, H2, sigHlim1, sigFlim1, sigHlim2, sigFlim2

(250, 240, 570, 450.00, 550, 432.00)

In [506]:
mH, mF, c, NFO = 6, 6, 1, 4e6

NHO1, NHO2 = 30*H1**2.4, 30*H2**2.4
Lh = 8*Ca*Kng*L
tmp = 60*Lh*c*(T1**3*t1 + T2**3*t2)/(t1+t2)
NHE1q, NHE2q, NHE1s, NHE2s = n_sh1*tmp, n_sh2*tmp, n_sh2*tmp, n_sh3*tmp
tmp = 60*Lh*c*(T1**mF*t1 + T2**mF*t2)/(t1+t2)
NFE1q, NFE2q, NFE1s, NFE2s = n_sh1*tmp, n_sh2*tmp, n_sh2*tmp, n_sh3*tmp
NHO1, NHO2, Lh, np.array([NHE1q, NHE2q, NHE1s, NHE2s, NFE1q, NFE2q, NFE1s, NFE2s])/1e9


(17067789.40,
 15474913.67,
 16640,
 array([2.11, 0.65, 0.65, 0.21, 1.83, 0.57, 0.57, 0.18]))

In [507]:
fh = lambda nho, nhe: (nho/nhe)**(1/mH)
ff = lambda nfe: (NFO/nfe)**(1/mF)
KHL1q, KHL2q, KHL1s, KHL2s = fh(NHO1,NHE1q), fh(NHO2,NHE2q), fh(NHO1,NHE1s), fh(NHO2,NHE2s)
KFL1q, KFL2q, KFL1s, KFL2s = ff(NFE1q), ff(NFE2q), ff(NFE1s), ff(NFE2s)
KHL1q, KHL2q, KHL1s, KHL2s, KFL1q, KFL2q, KFL1s, KFL2s

(0.45, 0.54, 0.54, 0.65, 0.36, 0.44, 0.44, 0.53)

In [508]:
cond = lambda k: 1 if k < 1 else k
KFC = 1 # one direction rotation
KHL1q, KHL2q, KHL1s, KHL2s, KFL1q, KFL2q, KFL1s, KFL2s = cond(KHL1q), cond(KHL2q), cond(KHL1s), cond(KHL2s), cond(KFL1q), cond(KFL2q), cond(KFL1s), cond(KFL2s)
KHL1q, KHL2q, KHL1s, KHL2s, KFL1q, KFL2q, KFL1s, KFL2s

sigH1q, sigH1s = sigHlim1/SH*KHL1q, sigHlim1/SH*KHL1s
sigH2q, sigH2s = sigHlim2/SH*KHL2q, sigHlim2/SH*KHL2s
sigF1q, sigF1s = sigFlim1/SF*KFL1q*KFC, sigFlim1/SF*KFL1s*KFC
sigF2q, sigF2s = sigFlim2/SF*KFL2q*KFC, sigFlim2/SF*KFL2s*KFC
sigH1q, sigH1s, sigH2q, sigH2s, sigF1q, sigF1s, sigF2q, sigF2s

(518.18, 518.18, 500.00, 500.00, 257.14, 257.14, 246.86, 246.86)

In [509]:
avg = lambda a, b: (a+b)/2
sigH1, sigH2 = avg(sigH1q,sigH2q), avg(sigH1s,sigH2s)
sigF1, sigF2 = avg(sigF1q,sigF2q), avg(sigF1s,sigF2s)
sigH1, sigH2, sigF1, sigF2, 1.25*min([sigH1q,sigH2q]), 1.25*min([sigH1s,sigH2s])

(509.09, 509.09, 252.00, 252.00, 625.00, 625.00)

In [510]:
sigH = sigH1
Ka, psiba, uq, us = 43, 0.3, u_1, u_2
psibdq = 0.53*psiba*(uq+1)
psibds = 0.53*psiba*(us+1)
fh5 = interp1d([.2,.4,.6,.8,1,1.2,1.4,1.6],[1.01,1.02,1.03,1.05,1.07,1.1 ,1.13,1.16])
fh4 = interp1d([.2,.4,.6,.8,1,1.2,1.4,1.6],[1.01,1.03,1.05,1.08,1.11,1.13,1.17,1.21])
ff5 = interp1d([.2,.4,.6,.8,1,1.2,1.4,1.6],[1.02,1.05,1.08,1.12,1.16,1.22,1.28,1.37])
ff4 = interp1d([.2,.4,.6,.8,1,1.2,1.4,1.6],[1.03,1.06,1.12,1.17,1.23,1.3 ,1.38,1.45])
KHbq, KHbs = float(fh5(psibdq)), float(fh4(psibds))
KFbq, KFbs = float(ff5(psibdq)), float(ff4(psibds))
psibdq, psibds, KHbq, KHbs, KFbq, KFbs

(0.67, 0.65, 1.04, 1.06, 1.09, 1.13)

In [511]:
# find center distance
awq = Ka*(uq+1)*(T_sh1*KHbq/sigH**2/uq/psiba)**(1/3)
aws = Ka*(us+1)*(T_sh2*KHbs/sigH**2/us/psiba)**(1/3)
awq, aws

(83.12, 119.85)

In [512]:
awq = 85; aws = 120
mq = np.array([.01,.02])*awq
ms = np.array([.01,.02])*aws
mq, ms

(array([0.85, 1.7 ]), array([1.2, 2.4]))

In [513]:
mq = 1.5; ms = 2; bet = 14
z1q = 2*awq*cos(bet*np.pi/180) / (mq*(uq+1))
z1s = 2*aws*cos(bet*np.pi/180) / (ms*(us+1))
z1q, z1s

(26.01, 28.42)

In [514]:
z1q, z1s = ceil(z1q), ceil(z1s)
z2q, z2s = uq*z1q, us*z1s
bq, bs = psiba*awq, psiba*aws
z2q, z2s, bq, bs

(87.17, 89.82, 25.50, 36.00)

In [4]:
bw = psiba*aw
# fig, [ax1,ax2] = plt.subplots(nrows=2)
# ax1.plot(m*(z1+z2)/2/aw)
# ax2.plot(bet)
# disp(np.arccos(min(m*(z1+z2)/2/aw))*180/np.pi)
# disp(z1)
# disp(bw,m*(z1+z2)/aw)
bet = np.arccos(m*(z1+z2)/2/aw)
d1 = dw1 = m*z1/np.cos(bet)
d2 = dw2 = m*z2/np.cos(bet)
da1 = d1+2*(1+0.3)*m
da2 = d2+2*(1-0.3)*m
df1 = d1-(2.5-2*0.3)*m
df2 = d2-(2.5-2*-0.3)*m
db1 = d1*np.cos(alp*np.pi/180)
db2 = d2*np.cos(alp*np.pi/180)
alp_t = alp_tw = np.arctan(np.tan(alp*np.pi/180)/np.cos(bet))
v = np.pi*d1*n_sh1/6e4
zv = 0.85*v**.1
ys = 1.08-0.0695*np.log(m)
sigHH, sigFF1, sigFF2 = sigH*zv, sigF1*1.1*ys, sigF2*1.1*ys
disp(np.round([d1,d2,da1,da2,df1,df2,db1,db2,
                alp_t*180/np.pi,bet*180/np.pi,v,zv,ys,
               sigHH, sigFF1, sigFF2],2))

array([[ 41.67, 208.33,  45.57, 210.43,  38.82, 203.68,  39.15, 195.77,
         20.53,  13.59,   6.39,   1.02,   1.05, 520.93, 297.51, 285.61]])

In [5]:
bet_b = np.arctan(np.cos(alp_t)*np.tan(bet))
zm, zh = 274, np.sqrt(2*np.cos(bet_b)/np.sin(2*alp_tw))
epbet = bw*np.sin(bet)/m/np.pi
epalp = (np.sqrt(da1**2-db1**2)+np.sqrt(da2**2-db2**2)-2*aw*np.sin(alp_tw))/(2*np.pi*m*np.cos(alp_t)/np.cos(bet))
disp(bet_b*180/np.pi,zh,epalp,epbet)
ze = epalp**-0.5
kh, kf, mn = 1.07*1.1*1.108, 1.18*1.29*1.2558, m*np.cos(bet)
sigHc = zm*zh*ze*np.sqrt(2*T_sh1*kh*(u_hg+1)/bw/u_hg/dw1**2)
ye, yb = epalp**-1, 1-bet*180/np.pi/140
zv1, zv2, yf1, yf2 = z1/np.cos(bet)**3, z2/np.cos(bet)**3, 3.54, 3.63
sigF1 = 2*T_sh1*kf*ye*yb*yf1/bw/dw1/mn
sigF2 = sigF1*yf2/yf1
Ft = 2*T_sh1/dw1
Fr = Ft*np.tan(alp_tw)
Fa = Ft*np.tan(bet)
disp(zv1,zv2,sigF1,sigF2)

array([12.76,  1.72,  1.41,  3.12])

array([ 29.4 , 147.01, 114.11, 117.01])

## Shaft design

In [9]:
# Fr,bet,dw1,dw2,T_sh1,T_sh2 = 1088.95,10.141*np.pi/180,32,128,21575.73,83127.84
# Fr of chain drive (chap 2)
# bet = beta_w (chap 3)
# dw1, dw2 (chap 3)
# t_sh1, t_sh2 (chap 1)
Ft1 = Ft2 = 2*T_sh1/dw1
Fr1 = Fr2 = Ft1*np.tan(alp*np.pi/180)/np.cos(bet)
Fa1 = Fa2 = Ft1*np.tan(bet)
r12, r21 = -dw1/2, dw2/2
hr12, cb12, cq1, hr21, cb21, cq2 = 1,1,1,-1,-1,-1
Fx12 = r12/abs(r12)*cq1*cb12*Ft1
Fx21 = r21/abs(r21)*cq2*cb21*Ft2
Fy12 = -r12/abs(r12)*Ft1*np.tan(alp_t)/np.cos(bet)
Fy21 = -r21/abs(r21)*Ft2*np.tan(alp_t)/np.cos(bet)
Fz12 = cq1*cb12*hr12*Ft1*np.tan(bet)
Fz21 = cq2*cb21*hr21*Ft2*np.tan(bet)
Fy22 = Fr*np.sin(210*np.pi/180)
Fx22 = Fr*np.cos(210*np.pi/180)
d1 = (T_sh1/.2/15)**(1/3)
d2 = (T_sh2/.2/30)**(1/3)
disp(d1,d2)
# d1, d2, bO1, bO2 = 32,28,20,18
d1, d2, bO1, bO2 = 35,40,21,23

lm13,lm12,lm22,lm23,k1,k2,k3,hn= 1.5*d1,1.5*d1,1.5*d2,1.5*d2,10,8,15,18
disp(np.round([lm13,lm12,lm22,lm23],2))
l22 = -(.5*(lm22+bO2)+k3+hn)
l23 = 0.5*(lm23+bO2)+k1+k2
l21 = 2*l23
l12 = -(.5*(lm12+bO1)+k3+hn)
l13 = 0.5*(lm13+bO1)+k1+k2
l11 = 2*l13
C2A2, A2D2, D2B2, A2B2 = -l22, l23, l23, l21
B1C1, A1D1, D1B1, A1B1 = -l12, l13, l13, l11
disp(l12,l13,l11,l22,l23,l21)
# disp(r12,r21,C2A2, A2D2, D2B2, A2B2,B1C1, A1D1, D1B1, A1B1)
disp(Ft1,Fr1,Fa1,Fx12,Fy12,Fz12,Fx21,Fy21,Fz21,Fx22,Fy22)

array([25.55, 34.1 ])

array([[52.5, 52.5, 60. , 60. ]])

array([-69.75,  54.75, 109.5 , -74.5 ,  59.5 , 119.  ])

array([ 2402.28,   899.55,   580.75, -2402.28,   925.46,   580.75,
        2402.28,  -925.46,  -580.75, -2320.05, -1339.48])

In [10]:
from sympy import *
rA2y, rA2x, rB2y, rB2x, rA1y, rA1x, rB1y, rB1x = symbols('r_A2y, r_A2x, r_B2y, r_B2x, r_A1y, r_A1x, r_B1y, r_B1x')
eq1 = rA2x + rB2x + Fx21 + Fx22
eq2 = C2A2*rA2x + (C2A2+A2D2)*Fx21 + (C2A2+A2B2)*rB2x
rA2x,rB2x = np.array(list(linsolve([eq1,eq2],[rA2x,rB2x]))[0],dtype='float')
disp((C2A2+A2D2)*Fx21,C2A2,A2B2)
eq1 = rA2y + rB2y + Fy21 + Fy22
eq2 = C2A2*rA2y + (C2A2+A2D2)*Fy21 + (C2A2+A2B2)*rB2y + dw2*Fz21/2
rA2y,rB2y = np.array(list(linsolve([eq1,eq2],[rA2y,rB2y]))[0],dtype='float')
# disp((C2A2+A2D2)*Fy21+dw2*Fz21/2,C2A2,A2D2,Fy21,dw2,Fz21)
eq1 = rA1x + rB1x + Fx12
eq2 = A1D1*Fx12 + A1B1*rB1x
rA1x,rB1x = np.array(list(linsolve([eq1,eq2],[rA1x,rB1x]))[0],dtype='float')
# disp(A1D1*Fx12)
eq1 = rA1y + rB1y + Fy12
eq2 = A1D1*Fy12 + A1B1*rB1y + dw1*Fz12/2
rA1y,rB1y = np.array(list(linsolve([eq1,eq2],[rA1y,rB1y]))[0],dtype='float')
disp(A1D1*Fy12+dw1*Fz12/2,A1D1,Fy12,dw1,Fz12)
MxA1, MxD1, MxB1, MxC1 = 0, A1D1*rA1x, 0, 0
MyA1, MyD1m, MyD1p, MyB1, MyC1 = 0, A1D1*rA1y, A1D1*rA1y - dw1*Fz12/2, A1B1*rA1y + D1B1*Fy12 - dw1*Fz12/2, 0
MxC2, MxA2, MxD2, MxB2 = 0, C2A2*Fx22, (C2A2+A2D2)*Fx22 + A2D2*rA2x, 0
MyC2, MyA2, MyD2m, MyD2p, MyB2 = 0, C2A2*Fy22, (C2A2+A2D2)*Fy22 + A2D2*rA2y, D2B2*rB2y,0
disp(Fy21+Fy22)
eqM = lambda mx, my, t: np.sqrt(mx**2 + my**2 + 0.75*t**2)
eq = lambda mx, my, t: np.sqrt(mx**2 + my**2)
MeA1 = eqM(MxA1,MyA1,0)
MeD1m = eqM(MxD1,MyD1m,T_sh1)
MeD1p = eqM(MxD1,MyD1p,T_sh1)
MeB1 = eqM(MxB1,MyB1,T_sh1)
MeC1 = eqM(MxC1,MyC1,T_sh1)
MeC2 = eqM(MxC2,MyC2,T_sh2)
MeA2 = eqM(MxA2,MyA2,T_sh2)
MeD2m = eqM(MxD2,MyD2m,T_sh2)
MeD2p = eqM(MxD2,MyD2p,T_sh2)
MeB2 = eqM(MxB2,MyB2,0)
MA1 = eq(MxA1,MyA1,0)
MD1m = eq(MxD1,MyD1m,T_sh1)
MD1p = eq(MxD1,MyD1p,T_sh1)
MB1 = eq(MxB1,MyB1,T_sh1)
MC1 = eq(MxC1,MyC1,T_sh1)
MC2 = eq(MxC2,MyC2,T_sh2)
MA2 = eq(MxA2,MyA2,T_sh2)
MD2m = eq(MxD2,MyD2m,T_sh2)
MD2p = eq(MxD2,MyD2p,T_sh2)
MB2 = eq(MxB2,MyB2,0)

Mes = np.array([MeA1,MeD1m,MeD1p,MeB1,MeC1,MeC2,MeA2,MeD2m,MeD2p,MeB2])
f = interp1d([30,50],[67,55])
ps1,ps2 = f([d1,d2])
sig = np.array([ps1,ps1,ps1,ps1,ps1,ps2,ps2,ps2,ps2,ps2])
# disp(dw1*Fz12/2,dw2*Fz21/2)
disp(rA1x,rA1y,rB1x,rB1y,rA2x,rA2y,rB2x,rB2y)
# disp(MeA1,MeD1m,MeD1p,MeB1,MeC1,MeC2,MeA2,MeD2m,MeD2p,MeB2)
# disp(MA1,MD1m,MD1p,MB1,MC1,MC2,MA2,MD2m,MD2p,MB2)
# np.array((Mes/0.1/sig)**(1/3))
# disp(rA1x,Fx12,rB1x,0,rA1y,Fy12,rB1y,0)
# disp(Fx22,rA2x,Fx21,rB2x,Fy22,rA2y,Fy21,rB2y)
disp(MxA1, MxD1, MxB1, MxC1)
disp(MyA1, MyD1m, MyD1p, MyB1, MyC1)
disp(MxC2, MxA2, MxD2, MxB2)
disp(MyC2, MyA2, MyD2m, MyD2p, MyB2)
disp(T_sh1,T_sh2)

array([321905.88,     74.5 ,    119.  ])

array([62767.9 ,    54.75,   925.46,    41.67,   580.75])

array([-2264.94])

array([ 1201.14,  -352.24,  1201.14,  -573.22,  2571.38,  2132.43,
       -2653.61,   132.51])

array([    0.  , 65762.49,     0.  ,     0.  ])

array([     0.  , -19284.96, -31383.95,      0.  ,      0.  ])

array([      0.  , -172843.64, -157889.73,       0.  ])

array([     0.  , -99791.32, -52610.71,   7884.23,      0.  ])

array([ 50047.56, 237825.99])

## bearing design

In [59]:
X1,Y1,X2,Y2,lh,m1,m2 = .6,1.07,1.66,1,19200,3,3
Q1 = X1*abs(rA1y)+Y1*abs(Fz12)
Q2 = X2*abs(rB2y)+Y1*abs(Fz21)
Cd1 = Q1*L1**(1/m1)
Cd2 = Q2*L2**(1/m2)
FrA1 = np.sqrt(rA1x**2+rA1y**2)
FrB1 = np.sqrt(rB1x**2+rB1y**2)
FrA2 = np.sqrt(rA2x**2+rA2y**2)
FrB2 = np.sqrt(rB2x**2+rB2y**2)
ratioA1 = abs(Fz12)/FrA1
ratioB1 = abs(Fz12)/FrB1
ratioA2 = abs(Fz12)/FrA2
ratioB2 = abs(Fz12)/FrB2
disp(ratioA1, ratioB1, ratioA2,ratioB2)
disp(FrA1,FrB1,FrA2,FrB2)/1e3
d2,D2,r2,C2,Co2=40,90,2.5,31.9,21.7
d1,D1,r1,C1,Co1=35,80,2.5,33.4,25.2
ratio1,ratio2=Fz12/Co1/1e3,Fz12/Co2/1e3
f1 = interp1d([.014,.028,.057,.085,.11,.17,.29,.43,.57],
                 [.3,.34,.37,.41,.45,.48,.52,.54,.54])
f2 = interp1d([.014,.028,.056,.084,.11,.17,.28,.42,.56],
               [.19,.22,.26,.28,.3,.34,.38,.42,.44])
e1, e2 = f1(ratio1),f2(ratio2)
# disp(e1,e2)

f1 = interp1d([.014,.028,.057,.085,.11,.17,.29,.43,.57],
                 [2.3,1.99,1.71,1.55,1.45,1.31,1.15,1.04,1])
f2 = interp1d([.014,.028,.056,.084,.11,.17,.28,.42,.56],
               [1.81,1.62,1.46,1.34,1.22,1.13,1.04,1.01,1])
X1,X2,Y1,Y2 = 0.56,0.45,f1(ratio1),f2(ratio2)
disp(X1,X2,Y1,Y2 )
FsA1 = e1*FrA1
FsB1 = e1*FrB1
FaA1 = FsB1 - Fz12
FaB1 = FsA1 + Fz12
FatA1, FatB1, FatA2, FatB2 = FsA1, FaB1, Fz12, Fz12
QA1 = (X1*FrA1 + Y1*FatA1)
QB1 = (X1*FrB1 + Y1*FatB1)
QA2 = (X2*FrA2 + Y2*FatA2)
QB2 = (X2*FrB2 + Y2*FatB2)
if QA1<QB1: Q1 = QB1
else: Q1 = QA1
if QA2<QB2: Q1 = QB2
else: Q2 = QA2
Qe1 = Q1*(T1**3*t1/(t1+t2)+T2**3*t2/(t1+t2))**(1/3)
Qe2 = Q2*(T1**3*t1/(t1+t2)+T2**3*t2/(t1+t2))**(1/3)
L1 = lh*60*n_sh1*1e-6/2
L2 = lh*60*n_sh2*1e-6/2
Cd1 = Qe1*L1**(1/3)
Cd2 = Qe2*L2**(1/3)
f = interp2d([320,400,500,630,800,1000,1250,1600,2000,2500,3200,4000],
            [8000,12500,16000,20000,25000,32000,40000,50000],
            np.array([[5.75,6.2,6.7,7.23,7.81,8.43,9.11,9.83,10.6,11.5,12.4,13.4],
                    [6.2,6.7,7.23,7.81,8.43,9.11,9.83,10.6,11.5,12.4,13.4,14.5],
                      [6.7,7.23,7.81,8.41,9.11,9.83,10.6,11.5,12.4,13.4,14.5,15.6],
                     [7.23,7.81,8.43,9.11,9.83,10.6,11.5,12.4,13.4,14.5,15.6,16.8],
                     [7.81,8.43,9.11,9.83,10.6,11.5,12.4,13.4,14.5,15.6,16.8,18.2],
                     [8.43,9.11,9.83,10.6,11.5,12.4,13.4,14.5,15.6,16.8,18.2,19.6],
                     [9.11,9.83,10.6,11.5,12.4,13.4,14.5,15.6,16.8,18.2,19.6,21.2],
                     [9.83,10.6,11.5,12.4,13.4,14.5,15.6,16.8,18.2,19.6,21.2,22.9]]))
ra1,ra2 = f(n_sh1,lh/2), f(n_sh2,lh/2)
C1 = Qe1*ra1
C2 = Qe2*ra2
XO,YO = .5,.47
Qt11 = XO*Fz12 + YO*FatB1
Qt12 = Fz12
if Qt11<Qt12: Qt1 = Qt12
else: Qt1 = Qt11
XO,YO = .6,.5
Qt21 = XO*Fz12 + YO*FatB2
Qt22 = Fz12
if Qt21<Qt22: Qt2 = Qt22
else: Qt2 = Qt21
disp(ra1,ra2)
disp(Qe1,Qe2,C1,C2,Cd1,Cd2,Qt1,Qt2)/1e3

array([0.46, 0.44, 0.17, 0.22])

array([1.25, 1.33, 3.34, 2.66])

array([0.56, 0.45, 2.1 , 1.64])

array([[12.39],
       [ 7.25]])

array([ 2.17,  1.88, 26.84, 13.66, 25.79, 13.12,  0.76,  0.64])